### Extracting Systematics using Geometric Subpixelisation at NSIDE = 2048



In [ ]:
import numpy as np
from astropy.io import fits
import healpy as hp
import pandas as pd
import matplotlib.path as mplPath
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict

In [ ]:
def raDec2thetaPhi(ra, dec):
    return (0.5 * np.pi - np.deg2rad(dec)), (np.deg2rad(ra))

NSIDE = 2048
NPIX = hp.nside2npix(NSIDE)

### Retrieving the relevant CCD Data

In [ ]:
decamCCD = fits.open('../../bricks_data/ccds-annotated-decam-dr9.fits')
mosaicCCD = fits.open('../../bricks_data/ccds-annotated-mosaic-dr9.fits')
bassCCD = fits.open('../../bricks_data/ccds-annotated-90prime-dr9.fits')
dataDecam = decamCCD[1].data
dataMosaic = mosaicCCD[1].data
dataBass = bassCCD[1].data


### Extracting CCD Informations from the entire Sample and Concatenating them together

In [ ]:
# Extracting Corners Only needed for visualisation

ra0 = np.concatenate((dataDecam.field('ra0'), dataMosaic.field('ra0'), dataBass.field('ra0')), axis=0)
dec0 = np.concatenate((dataDecam.field('dec0'), dataMosaic.field('dec0'), dataBass.field('dec0')), axis=0)

ra1 = np.concatenate((dataDecam.field('ra1'), dataMosaic.field('ra1'), dataBass.field('ra1')), axis=0)
dec1 = np.concatenate((dataDecam.field('dec1'), dataMosaic.field('dec1'), dataBass.field('dec1')), axis=0)

ra2 = np.concatenate((dataDecam.field('ra2'), dataMosaic.field('ra2'), dataBass.field('ra2')), axis=0)
dec2 = np.concatenate((dataDecam.field('dec2'), dataMosaic.field('dec2'), dataBass.field('dec2')), axis=0)

ra3 = np.concatenate((dataDecam.field('ra3'), dataMosaic.field('ra3'), dataBass.field('ra3')), axis=0)
dec3 = np.concatenate((dataDecam.field('dec3'), dataMosaic.field('dec3'), dataBass.field('dec3')), axis=0)

#airmass, fwhm, skysb, exptime, mjd, meansky, galdepths,

#Extracting systematics
filter_colour = np.concatenate((dataDecam.field('filter'), dataMosaic.field('filter'), dataBass.field('filter')),
                               axis=0)
exptime = np.concatenate((dataDecam.field('exptime'), dataMosaic.field('exptime'), dataBass.field('exptime')), axis=0)
airmass = np.concatenate((dataDecam.field('airmass'), dataMosaic.field('airmass'), dataBass.field('airmass')), axis=0)
fwhm = np.concatenate((dataDecam.field('fwhm'), dataMosaic.field('fwhm'), dataBass.field('fwhm')), axis=0)
seeing = fwhm * 0.262
ccdskysb = np.concatenate((dataDecam.field('ccdskysb'), dataMosaic.field('ccdskysb'), dataBass.field('ccdskysb')),
                          axis=0)
meansky = np.concatenate((dataDecam.field('meansky'), dataMosaic.field('meansky'), dataBass.field('meansky')), axis=0)
galdepth = np.concatenate((dataDecam.field('galdepth'), dataMosaic.field('galdepth'), dataBass.field('galdepth')),
                          axis=0)
psfdepth = np.concatenate((dataDecam.field('psfdepth'), dataMosaic.field('psfdepth'), dataBass.field('psfdepth')),
                          axis=0)
psfnorm_mean = np.concatenate(
    (dataDecam.field('psfnorm_mean'), dataMosaic.field('psfnorm_mean'), dataBass.field('psfnorm_mean')), axis=0)
gausspsfdepth = np.concatenate(
    (dataDecam.field('gausspsfdepth'), dataMosaic.field('gausspsfdepth'), dataBass.field('gausspsfdepth')), axis=0)


### Now, trying to actually get all ccds per pixel
Begin by loading the Dictionary mapping pixels to CCDs. Afterwards, iterating through the ccds for a given pixel.
Depending on the systematics, we have to break systematics down into the different bands.
Airmass is simply averaged across all CCDs, exposure times are calculated on a per CCD basis.


### Import subpixel2ccd mapping


In [ ]:
# Use this cell to simply import an existing subpixel2ccd mapping

# For DECAM, BASS, MzLS
with open('../../bricks_data/pixel2ccd_2048_non_inclusive.pickle', 'rb') as f:
    pixel2ccd_dict = pickle.load(f)
    f.close()

###  Get systematics value across every subpixel and average --> this is where the procedure differs from prev
- Iterate through all pixels
- Get systematic value at pixel
- average for pixel

### Adapt this loop to no longer iterate over subpixels

In [ ]:
pixels_overall = pixel2ccd_dict.keys()
print(len(pixels_overall))

pixel2systematics_dict = defaultdict(list)

# Find subpixels for the given pixel
for i, sample_pixel in enumerate(pixels_overall):
    subpixels_per_pixel = pixel2subpixel_dict[sample_pixel]

    airmass_aggregate = 0

    seeing_aggregate_g = 0
    seeing_aggregate_r = 0
    seeing_aggregate_z = 0

    ccdskysb_aggregate_g = 0
    ccdskysb_aggregate_r = 0
    ccdskysb_aggregate_z = 0

    exptime_aggregate_g = 0
    exptime_aggregate_r = 0
    exptime_aggregate_z = 0

    meansky_aggregate_g = 0
    meansky_aggregate_r = 0
    meansky_aggregate_z = 0

    galdepth_aggregate_g = 0
    galdepth_aggregate_r = 0
    galdepth_aggregate_z = 0

    psfdepth_aggregate_g = 0
    psfdepth_aggregate_r = 0
    psfdepth_aggregate_z = 0

    psfnorm_mean_aggregate_g = 0
    psfnorm_mean_aggregate_r = 0
    psfnorm_mean_aggregate_z = 0

    gausspsfdepth_aggregate_g = 0
    gausspsfdepth_aggregate_r = 0
    gausspsfdepth_aggregate_z = 0

    subpixels_covered = 0
    subpixels_covered_g = 0
    subpixels_covered_r = 0
    subpixels_covered_z = 0
    # Go through all 64 subpixels in the sample pixel
    for subpixel in subpixels_per_pixel:
        # Condition needed in case a subpixel is not covered by CCDs
        if subpixel not in subpixel2ccd_dict.keys():
            continue
        subpixels_covered += 1
        ccds_per_subpixel = subpixel2ccd_dict[subpixel]
        # Get values for singular systematics
        airmass_aggregate += airmass[ccds_per_subpixel].sum() / len(ccds_per_subpixel)

        # Get values for per band systematics

        mask_g = (filter_colour[ccds_per_subpixel] == 'g')
        mask_r = (filter_colour[ccds_per_subpixel] == 'r')
        mask_z = (filter_colour[ccds_per_subpixel] == 'z')

        expt = exptime[ccds_per_subpixel]
        exptime_g = expt[mask_g]
        exptime_r = expt[mask_r]
        exptime_z = expt[mask_z]

        see = seeing[ccds_per_subpixel]
        seeing_g = see[mask_g]
        seeing_r = see[mask_r]
        seeing_z = see[mask_z]

        # Sky background
        sb = ccdskysb[ccds_per_subpixel]
        ccdskysb_g = sb[mask_g]
        ccdskysb_r = sb[mask_r]
        ccdskysb_z = sb[mask_z]

        # Sky level
        msl = meansky[ccds_per_subpixel]
        meansky_g = msl[mask_g]
        meansky_r = msl[mask_r]
        meansky_z = msl[mask_z]

        # Galaxy Depth
        gd = galdepth[ccds_per_subpixel]
        galdepth_g = gd[mask_g]
        galdepth_r = gd[mask_r]
        galdepth_z = gd[mask_z]

        psfd = psfdepth[ccds_per_subpixel]
        psfdepth_g = psfd[mask_g]
        psfdepth_r = psfd[mask_r]
        psfdepth_z = psfd[mask_z]

        psfnorm = psfnorm_mean[ccds_per_subpixel]
        psfnorm_g = psfnorm[mask_g]
        psfnorm_r = psfnorm[mask_r]
        psfnorm_z = psfnorm[mask_z]

        gausspsfnorm = gausspsfdepth[ccds_per_subpixel]
        gausspsf_g = gausspsfnorm[mask_g]
        gausspsf_r = gausspsfnorm[mask_r]
        gausspsf_z = gausspsfnorm[mask_z]

        g_exp = mask_g.sum()
        if g_exp > 0:
            subpixels_covered_g += 1
            exptime_aggregate_g = exptime_g.sum() / g_exp
            ccdskysb_aggregate_g = ccdskysb_g.sum() / g_exp
            meansky_aggregate_g = meansky_g.sum() / g_exp
            galdepth_aggregate_g = galdepth_g.sum() / g_exp
            seeing_aggregate_g = seeing_g.sum() / g_exp
            psfdepth_aggregate_g = psfdepth_g.sum() / g_exp
            psfnorm_mean_aggregate_g = psfnorm_g.sum() / g_exp
            gausspsfdepth_aggregate_g = gausspsf_g.sum() / g_exp

        r_exp = mask_r.sum()
        if r_exp > 0:
            subpixels_covered_r += 1
            exptime_aggregate_r = exptime_r.sum() / r_exp
            ccdskysb_aggregate_r = ccdskysb_r.sum() / r_exp
            meansky_aggregate_r = meansky_r.sum() / r_exp
            galdepth_aggregate_r = galdepth_r.sum() / r_exp
            seeing_aggregate_r = seeing_r.sum() / r_exp
            psfdepth_aggregate_r = psfdepth_r.sum() / r_exp
            psfnorm_mean_aggregate_r = psfnorm_r.sum() / r_exp
            gausspsfdepth_aggregate_r = gausspsf_r.sum() / r_exp

        z_exp = mask_z.sum()
        if z_exp > 0:
            subpixels_covered_z += 1
            exptime_aggregate_z = exptime_z.sum() / z_exp
            ccdskysb_aggregate_z = ccdskysb_z.sum() / z_exp
            meansky_aggregate_z = meansky_z.sum() / z_exp
            galdepth_aggregate_z = galdepth_z.sum() / z_exp
            seeing_aggregate_z = seeing_z.sum() / z_exp
            psfdepth_aggregate_z = psfdepth_z.sum() / z_exp
            psfnorm_mean_aggregate_z = psfnorm_z.sum() / z_exp
            gausspsfdepth_aggregate_z = gausspsf_z.sum() / z_exp

    if i % 4567 == 0:
        print(int(i / 4567), '%')

    # Do not append to dictionary if less than 75% of subpixels per pixel are covered by CCDs
    if subpixels_covered < 48:
        continue

    systematics_per_pixel = []
    systematics_per_pixel.append(airmass_aggregate / subpixels_covered)

    systematics_per_pixel.append(ccdskysb_aggregate_g)  #/subpixels_covered_g)
    systematics_per_pixel.append(ccdskysb_aggregate_r)  #/subpixels_covered_r)
    systematics_per_pixel.append(ccdskysb_aggregate_z)  #/subpixels_covered_z)

    systematics_per_pixel.append(exptime_aggregate_g)  #/subpixels_covered_g)
    systematics_per_pixel.append(exptime_aggregate_r)  #/subpixels_covered_r)
    systematics_per_pixel.append(exptime_aggregate_z)  #/subpixels_covered_z)

    systematics_per_pixel.append(meansky_aggregate_g)  #/subpixels_covered_g)
    systematics_per_pixel.append(meansky_aggregate_r)  #/subpixels_covered_r)
    systematics_per_pixel.append(meansky_aggregate_z)  #/subpixels_covered_z)

    systematics_per_pixel.append(galdepth_aggregate_g)  #/subpixels_covered_g)
    systematics_per_pixel.append(galdepth_aggregate_r)  #/subpixels_covered_r)
    systematics_per_pixel.append(galdepth_aggregate_z)  #/subpixels_covered_z)

    systematics_per_pixel.append(seeing_aggregate_g)  #/subpixels_covered_g)
    systematics_per_pixel.append(seeing_aggregate_r)  #/subpixels_covered_r)
    systematics_per_pixel.append(seeing_aggregate_z)  #/subpixels_covered_z)

    systematics_per_pixel.append(psfdepth_aggregate_g)  #/subpixels_covered_g)
    systematics_per_pixel.append(psfdepth_aggregate_r)  #/subpixels_covered_r)
    systematics_per_pixel.append(psfdepth_aggregate_z)  #/subpixels_covered_z)

    systematics_per_pixel.append(psfnorm_mean_aggregate_g)  #/subpixels_covered_g)
    systematics_per_pixel.append(psfnorm_mean_aggregate_r)  #/subpixels_covered_r)
    systematics_per_pixel.append(psfnorm_mean_aggregate_z)  #/subpixels_covered_z)

    systematics_per_pixel.append(gausspsfdepth_aggregate_g)  #/subpixels_covered_g)
    systematics_per_pixel.append(gausspsfdepth_aggregate_r)  #/subpixels_covered_r)
    systematics_per_pixel.append(gausspsfdepth_aggregate_z)  #/subpixels_covered_z)

    # Also appending fraction of pixel covered to cut on it later
    systematics_per_pixel.append(subpixels_covered / 64)  #/subpixels_covered_z)

    #/subpixels_covered_z)

    pixel2systematics_dict[sample_pixel] = systematics_per_pixel


